<a href="https://colab.research.google.com/github/BouzCS/Sign-Language-Detection/blob/main/Sign_Language_Detection(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries & Unzip files

In [1]:
# Let`s import all packages that we may need:
import pandas as pd
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # preproc_data processing, CSV file I/O (e.g. pd.read_csv), preproc_data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 
from sklearn.model_selection import train_test_split # to split the preproc_data into two parts
from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline # pipeline making
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics # for the check the error and accuracy of the model
from sklearn.metrics import mean_squared_error,r2_score

import glob
import cv2
## for Deep-learing:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import itertools
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Dropout, Flatten

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!zip -FF "drive/My Drive/ISL_CSLRT_Corpus.zip" -O 'fixed.zip'

Streaming output truncated to the last 5000 lines.
 copying: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/2/what do you want to become (2) 12.jpg  (7984 bytes)
 copying: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/2/what do you want to become (2) 13.jpg  (7975 bytes)
 copying: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/2/what do you want to become (2) 14.jpg  (7917 bytes)
 copying: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/3/  (0 bytes)
 copying: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/3/become 01.jpg  (103603 bytes)
 copying: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/3/become 02.jpg  (104251 bytes)
 copying: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/3/become 03.jpg  (104238 bytes)
 copying: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/3/become 04.jpg  (105072 bytes)
 copying: ISL_CSLRT_Corpus/Frames_Sen

In [ ]:
!unzip /content/fixed.zip

Streaming output truncated to the last 5000 lines.
  inflating: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/2/what do you want to become (2) 08.jpg  
  inflating: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/2/what do you want to become (2) 09.jpg  
  inflating: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/2/what do you want to become (2) 10.jpg  
  inflating: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/2/what do you want to become (2) 11.jpg  
  inflating: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/2/what do you want to become (2) 12.jpg  
  inflating: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/2/what do you want to become (2) 13.jpg  
  inflating: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/2/what do you want to become (2) 14.jpg  
   creating: ISL_CSLRT_Corpus/Frames_Sentence_Level/what do you want to become/3/
  inflating: ISL_CSL

In [ ]:
!pip install glob2 tk pathlib datetime tqdm mediapipe

     |████████████████████████████████| 51 kB 341 kB/s 
     |████████████████████████████████| 32.7 MB 1.6 MB/s 
     |████████████████████████████████| 251 kB 41.1 MB/s 


In [ ]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y, landmark_z])

    return landmark_point

In [ ]:
# Important : Preprocessing :
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

In [ ]:
import cv2
import copy
import itertools
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
global_preproc_data = []
new_preproc_data=[]
hands_list=[]
# For static images:
IMAGE_FILES = ["/content/index.jpg"]
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.7) as hands:
  for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    try:
      for idx, hand_handedness in enumerate(results.multi_handedness):
        hands_list.append(hand_handedness.classification[0].label)
    except:
        pass
    hand_max_count = max(hands_list.count("Right"), hands_list.count("Left"))
    # Print handedness and draw hand landmarks on the image.
    print('Handedness:', results.multi_handedness)
    if not results.multi_hand_landmarks:
      continue
    image_height, image_width, _ = image.shape
    annotated_image = image.copy()
    for hand_landmarks in results.multi_hand_landmarks:
      if results.multi_hand_landmarks:
        for i in range(21):
            print(f'{mp_hands.HandLandmark(i).name}:')
            print(f'{hand_landmarks.landmark[mp_hands.HandLandmark(i).value]}')
            global_preproc_data.append(min(int(hand_landmarks.landmark[mp_hands.HandLandmark(i).value].x * image_width), image_width - 1))
            global_preproc_data.append(min(int(hand_landmarks.landmark[mp_hands.HandLandmark(i).value].y * image_width), image_width - 1))
            global_preproc_data.append(hand_landmarks.landmark[mp_hands.HandLandmark(i).value].z)
            new_preproc_data.append(global_preproc_data[i*3:(i*3)+3])

      preproc_data=np.array(pre_process_landmark(new_preproc_data))
      if "Right" in hands_list and "Left" in hands_list:
        df = df.append(pd.DataFrame(preproc_data.reshape(1,-1), columns=col))


      print(
          f'Index finger tip coordinates: (',
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
      )
      mp_drawing.draw_landmarks(
          annotated_image,
          hand_landmarks,
          mp_hands.HAND_CONNECTIONS,
          mp_drawing_styles.get_default_hand_landmarks_style(),
          mp_drawing_styles.get_default_hand_connections_style())
    cv2.imwrite(
        '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))
    # Draw hand world landmarks.
    if not results.multi_hand_world_landmarks:
      continue
    for hand_world_landmarks in results.multi_hand_world_landmarks:
      mp_drawing.plot_landmarks(
        hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)

Handedness: [classification {
  index: 1
  score: 0.8703872561454773
  label: "Right"
}
, classification {
  index: 0
  score: 0.8428965210914612
  label: "Left"
}
]
WRIST:
x: 0.17833852767944336
y: 0.8823860287666321
z: 5.63905302897183e-07

THUMB_CMC:
x: 0.28911691904067993
y: 0.8276882171630859
z: -0.02192401885986328

THUMB_MCP:
x: 0.36804354190826416
y: 0.7015381455421448
z: -0.02912747487425804

THUMB_IP:
x: 0.4202536344528198
y: 0.5894691348075867
z: -0.034711163491010666

THUMB_TIP:
x: 0.4598950743675232
y: 0.5044523477554321
z: -0.041468195617198944

INDEX_FINGER_MCP:
x: 0.28976738452911377
y: 0.5013773441314697
z: -0.02717367745935917

INDEX_FINGER_PIP:
x: 0.3117188513278961
y: 0.33055779337882996
z: -0.03909102827310562

INDEX_FINGER_DIP:
x: 0.32438045740127563
y: 0.23261630535125732
z: -0.04778371751308441

INDEX_FINGER_TIP:
x: 0.33350417017936707
y: 0.1532464325428009
z: -0.05600178986787796

MIDDLE_FINGER_MCP:
x: 0.22397425770759583
y: 0.47533559799194336
z: -0.0298851858

NameError: ignored

In [ ]:
HAND_DICT ={
    '0': "WRIST",
    "1": "THUMB_CMC",
    "2": 'THUMB_MCP',
    "3": 'THUMB_IP',
    "4": 'THUMB_TIP',
    "5": 'INDEX_FINGER_MCP',
    "6": "INDEX_FINGER_PIP",
    "7": 'INDEX_FINGER_DIP',
    "8": "INDEX_FINGER_TIP",
    "9": "MIDDLE_FINGER_MCP",
    "10": "MIDDLE_FINGER_PIP",
    "11": "MIDDLE_FINGER_DIP",
    "12": 'MIDDLE_FINGER_TIP',
    "13": 'RING_FINGER_MCP',
    "14": "RING_FINGER_PIP",
    "15": 'RING_FINGER_DIP',
    "16": "RING_LFINGER_TIP",
    "17": 'PINKY_MCP',
    "18": 'PINKY_PIP',
    "19": "PINKY_DIP",
    "20": "PINKY_TIP"
    }

In [ ]:
dfx = pd.read_excel("/content/ISL_CSLRT_Corpus/corpus_csv_files/ISL_CSLRT_Corpus_word_details.xlsx")
len(dfx['Word'].unique())

114

# Augumentation images data

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import tqdm
import os

preproc_datagen = ImageDataGenerator(        
        rotation_range = 30,
        shear_range = 0.2,
        zoom_range = 0.4,
        horizontal_flip = True,
        brightness_range = (0.5, 1.5))

path="/content/ISL_CSLRT_Corpus/Frames_Word_Level"

for file in tqdm.tqdm(glob.glob("/content/ISL_CSLRT_Corpus/Frames_Word_Level/*/*")):
    file_name=file.split("/")[-1]
    base_name=os.path.dirname(file).split("/")[-1]
    img = load_img(path+"/"+base_name+"/"+file_name)  
    x = img_to_array(img) 
    # Reshape the input image 
    x = x.reshape((1, ) + x.shape)  
    i = 0
    # generate 5 new augmented images 
    for batch in preproc_datagen.flow(x, batch_size = 1, 
                      save_to_dir ='/content/ISL_CSLRT_Corpus/Frames_Word_Level'+'/'+base_name,  
                      save_prefix = base_name, save_format ='jpeg'):
        i += 1
        if i > 11: 
            break

100%|██████████| 1036/1036 [2:28:30<00:00,  8.60s/it]


# Create path data

In [ ]:
import glob, os
file_name=[]
base_path=[]
for file in glob.glob("/content/ISL_CSLRT_Corpus/Frames_Word_Level/*/*"):
    
    file_name.append(file.split("/")[-1])
    base_path.append(file)
import pandas as pd

path_data= pd.DataFrame({"file_name":file_name,"base_path":base_path},columns=["file_name","base_path"])


In [ ]:
path_data

,file_name,base_path
0,DILEMMA_0_7608.jpeg,/content/ISL_CSLRT_Corpus/Frames_Word_Level/DI...
1,DILEMMA (1).jpg,/content/ISL_CSLRT_Corpus/Frames_Word_Level/DI...
2,DILEMMA_0_8207.jpeg,/content/ISL_CSLRT_Corpus/Frames_Word_Level/DI...
3,DILEMMA_0_7306.jpeg,/content/ISL_CSLRT_Corpus/Frames_Word_Level/DI...
4,DILEMMA_0_3016.jpeg,/content/ISL_CSLRT_Corpus/Frames_Word_Level/DI...
...,...,...
13258,HURT_0_9105.jpeg,/content/ISL_CSLRT_Corpus/Frames_Word_Level/HU...
13259,HURT_0_7064.jpeg,/content/ISL_CSLRT_Corpus/Frames_Word_Level/HU...
13260,HURT_0_9226.jpeg,/content/ISL_CSLRT_Corpus/Frames_Word_Level/HU...
13261,HURT_0_1425.jpeg,/content/ISL_CSLRT_Corpus/Frames_Word_Level/HU...


In [ ]:
path_data["base_path"] = path_data['base_path'].astype(str)

In [ ]:
col_one_list = path_data['base_path'].tolist()
#col_one_list = ["\\content\\" + i for i in col_one_list]
#col_one_list = [ i.replace("\\", "/")  for i in col_one_list]

In [ ]:
len(col_one_list)

13263

# Create images data

In [ ]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y, landmark_z])

    return landmark_point

In [ ]:
# Important : Preprocessing :
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

In [ ]:
import cv2
import pandas as pd
import numpy as np
import mediapipe as mp
import tqdm
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

zero = np.zeros(63)
col = np.load("/content/drive/MyDrive/col(1).npy")
df = pd.DataFrame(columns=col)
col=np.append(col,["label"])
# For static images:

IMAGE_FILES = col_one_list
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.64) as hands:
  for idx, file in tqdm.tqdm(enumerate(IMAGE_FILES)):
    hands_list=[]
    data = []
    temp_data = []
    label=os.path.dirname(file).split("/")[-1]


    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    try:
      for idx, hand_handedness in enumerate(results.multi_handedness):
        hands_list.append(hand_handedness.classification[0].label)
    except:
        pass
    hand_max_count = max(hands_list.count("Right"), hands_list.count("Left"))
    # Print handedness and draw hand landmarks on the image.
    #print('Handedness:', results.multi_handedness)
    if not results.multi_hand_landmarks:
      continue
    annotated_image = image.copy()
    for hand_landmarks in results.multi_hand_landmarks:
      if results.multi_hand_landmarks:
          for i in range(21):
            data.append(min(int(hand_landmarks.landmark[mp_hands.HandLandmark(i).value].x * image_width), image_width - 1))
            data.append(min(int(hand_landmarks.landmark[mp_hands.HandLandmark(i).value].y * image_width), image_width - 1))
            data.append(hand_landmarks.landmark[mp_hands.HandLandmark(i).value].z)
            temp_data.append(data[i*3:(i*3)+3])

    preproc_data=np.array(pre_process_landmark(temp_data))
      
    if "Right" in hands_list and "Left" in hands_list:
      preproc_data = np.concatenate((preproc_data,[label]), axis=None)
      df = df.append(pd.DataFrame(preproc_data.reshape(1,-1), columns=col))

    elif ("Right" in hands_list and "Left" not in hands_list):
      if hand_max_count==1:
        preproc_data = np.concatenate((preproc_data, zero), axis=None)
        preproc_data = np.concatenate((preproc_data,[label]), axis=None)

        df =df.append(pd.DataFrame(preproc_data.reshape(1,-1), columns=col))

      else:
        preproc_data = np.concatenate((preproc_data[:63], zero), axis=None)
        preproc_data = np.concatenate((preproc_data,[label]), axis=None)

        df =df.append(pd.DataFrame(preproc_data.reshape(1,-1), columns=col))       

    elif ("Right" not in hands_list and "Left" in hands_list):
      if hand_max_count==1:

        preproc_data = np.concatenate((zero, preproc_data), axis=None)
        preproc_data = np.concatenate((preproc_data,[label]), axis=None)

        df =df.append(pd.DataFrame(preproc_data.reshape(1,-1), columns=col))

      else:

        preproc_data = np.concatenate((zero, preproc_data[:63]), axis=None)
        preproc_data = np.concatenate((preproc_data,[label]), axis=None)

        df =df.append(pd.DataFrame(preproc_data.reshape(1,-1), columns=col))



      
          



13263it [42:51,  5.16it/s]


In [ ]:
df.to_csv("df_final_preproc_10k.csv")

In [ ]:
!cp /content/df_final_preproc_10k.csv /content/drive/MyDrive

In [3]:
df=pd.read_csv("/content/drive/MyDrive/df_final_preproc_10k.csv")
df

,Unnamed: 0,R_WRIST_x,R_WRIST_y,R_WRIST_z,R_THUMB_CMC_x,R_THUMB_CMC_y,R_THUMB_CMC_z,R_THUMB_MCP_x,R_THUMB_MCP_y,R_THUMB_MCP_z,...,L_PINKY_PIP_x,L_PINKY_PIP_y,L_PINKY_PIP_z,L_PINKY_DIP_x,L_PINKY_DIP_y,L_PINKY_DIP_z,L_PINKY_TIP_x,L_PINKY_TIP_y,L_PINKY_TIP_z,label
0,0,0.0,0.0,1.885247e-10,-0.312500,-0.187500,-0.000142,-0.562500,-0.375000,-0.000279,...,-0.437500,-0.562500,-0.001127,-0.562500,-0.812500,-0.001255,-0.687500,-1.000000,-0.001305,DILEMMA
1,0,0.0,0.0,-4.304259e-09,-0.133333,0.300000,0.000040,-0.333333,0.466667,0.000053,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,DILEMMA
2,0,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.410256,0.051282,-0.000797,-0.461538,-0.025641,-0.000918,-0.512821,-0.153846,-0.000998,DILEMMA
3,0,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.600000,0.371429,-0.000371,-0.714286,0.371429,-0.000433,-0.828571,0.342857,-0.000473,DILEMMA
4,0,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.580645,0.451613,-0.000271,-0.709677,0.483871,-0.000332,-0.806452,0.483871,-0.000365,DILEMMA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11535,0,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.354167,-0.041667,-0.000742,0.395833,-0.041667,-0.000736,0.437500,-0.062500,-0.000700,HURT
11536,0,0.0,0.0,-1.419969e-10,0.152381,-0.085714,0.000056,0.342857,-0.076190,0.000051,...,0.504762,0.219048,-0.000495,0.590476,0.257143,-0.000540,0.666667,0.276190,-0.000538,HURT
11537,0,0.0,0.0,2.292120e-09,0.000000,-0.437500,0.000118,-0.125000,-0.687500,0.000204,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,HURT
11538,0,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.500000,-0.289474,-0.000603,0.605263,-0.315789,-0.000652,0.684211,-0.368421,-0.000662,HURT


In [4]:
df.drop("Unnamed: 0",axis=1,inplace=True)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df

,R_WRIST_x,R_WRIST_y,R_WRIST_z,R_THUMB_CMC_x,R_THUMB_CMC_y,R_THUMB_CMC_z,R_THUMB_MCP_x,R_THUMB_MCP_y,R_THUMB_MCP_z,R_THUMB_IP_x,...,L_PINKY_PIP_x,L_PINKY_PIP_y,L_PINKY_PIP_z,L_PINKY_DIP_x,L_PINKY_DIP_y,L_PINKY_DIP_z,L_PINKY_TIP_x,L_PINKY_TIP_y,L_PINKY_TIP_z,label
0,0.451097,0.514370,1.188836e-07,0.469132,0.487596,0.000110,0.475352,0.455454,-0.000505,0.473375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SORRY
1,0.515878,0.540605,1.321693e-07,0.518887,0.506151,-0.006976,0.508161,0.483509,-0.010528,0.492018,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SORRY
2,0.461494,0.559283,8.450694e-08,0.468477,0.519019,0.000079,0.464193,0.484450,-0.000635,0.453978,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SORRY
3,0.520546,0.539417,1.704826e-07,0.519644,0.501733,-0.005887,0.505783,0.479929,-0.009033,0.487484,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SORRY
4,0.468136,0.571441,8.413286e-08,0.463234,0.529842,0.002238,0.452160,0.503096,0.003676,0.437428,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SORRY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12075,0.633831,0.627656,2.343263e-07,0.616618,0.596139,-0.010197,0.593095,0.581696,-0.017861,0.571217,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HUNGRY
12076,0.533230,0.773937,2.599143e-07,0.534586,0.724046,-0.014066,0.523717,0.682694,-0.022719,0.509977,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HUNGRY
12077,0.475212,0.836951,1.282162e-07,0.473822,0.791402,-0.000566,0.459309,0.757602,-0.003369,0.443167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HUNGRY
12078,0.577192,0.721748,1.172388e-07,0.558289,0.699846,-0.003462,0.538887,0.699384,-0.006429,0.521990,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HUNGRY


#MLP Classifier

In [5]:

# define preproc_dataset
X=df.drop("label",axis=1)
y= df['label']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)


In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score, accuracy_score

GRID = [
    {
     'solver': ['adam'],
     'learning_rate_init': [0.0001],
     'max_iter': [250],
     'hidden_layer_sizes': [(500, 400, 300, 200, 100)],
     'activation': ['relu'],
     'alpha': [0.0001],
     }
]


model=MLPClassifier()

grid_search = GridSearchCV(estimator=model, param_grid=GRID,
                            n_jobs=-1, cv=2, refit=True, verbose=3, 
                            return_train_score=False)

grid_search.fit(X_train, y_train)
prediction = grid_search.predict(X_test)


%time


Fitting 2 folds for each of 1 candidates, totalling 2 fits


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 9.3 µs


In [13]:

print(grid_search.best_params_)
print('Score of best mlp_classifier :\t{}'.format(grid_search.score(X_test, y_test)))
print(classification_report(y_test, prediction))

{'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (500, 400, 300, 200, 100), 'learning_rate_init': 0.0001, 'max_iter': 250, 'solver': 'adam'}
Score of best mlp_classifier :	0.45156208978734574
                 precision    recall  f1-score   support

          A LOT       0.25      0.13      0.17        15
          ABUSE       0.43      0.23      0.30        13
         AFRAID       0.42      0.58      0.48        26
          AGREE       0.37      0.38      0.38        26
            ALL       0.17      0.06      0.09        16
          ANGRY       0.62      0.78      0.69        45
       ANYTHING       0.50      0.06      0.11        16
     APPRECIATE       0.33      0.30      0.32        20
            BAD       0.32      0.37      0.34        19
      BEAUTIFUL       0.08      0.08      0.08        12
         BECOME       0.36      0.38      0.37        21
            BED       0.23      0.24      0.23        21
          BORED       0.40      0.35      0.37       

In [ ]:
import pickle
# now you can save it to a file
with open('mlpclf.pkl', 'wb') as f:
    pickle.dump(grid_search, f)
%time
# and later you can load it
#with open('filename.pkl', 'rb') as f:
#    clf = pickle.load(f)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 9.54 µs


In [ ]:
!cp /content/mlpclf.pkl /content/drive/MyDrive/

# XGB Classifier

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [ ]:
params = {
        'min_child_weight': [5, 10],
        'gamma': [0.5,1.5,5],
        'subsample': [0.6, 0.8],
        'colsample_bytree': [0.6, 1.0],
        'max_depth': [3, 5]
        }

xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='multi:softmax',
                    silent=True, nthread=1)


folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train,y_train), verbose=3, random_state=1001 )

# Here we go
random_search.fit(X_train, y_train)

%time

In [ ]:
from sklearn.metrics import classification_report, f1_score, accuracy_score

print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
prediction = random_search.predict(X_test)
print('Score of best mlp_classifier :\t{}'.format(random_search.score(X_test, y_test)))
print(classification_report(y_test, prediction))

In [ ]:
# example of manually specifying the loss function for binary classification
from sklearn.model_selection import train_test_split, cross_val_predict
import xgboost as xgb
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score 

# define preproc_dataset
X=df.drop("label",axis=1)
y= df['label']
# create XGBoost instance with default hyper-parameters
xgb_estimator = xgb.XGBClassifier(objective='multi:softmax')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# fit the model
xgb_pipe = Pipeline([
        ('sc', StandardScaler()),     
        ('xgb', xgb_estimator) 
    ])
xgb_pipe.fit(X_train, y_train)
# summarize the model loss function
print('Accuracy on test preproc_data: {:.1f}%'.format(accuracy_score(y_test, xgb_pipe.predict(X_test))*100))
%time

In [ ]:
print('Accuracy on test preproc_data: {:.1f}%'.format(accuracy_score(y_test, xgb_pipe.predict(X_test))*100))


Accuracy on test preproc_data: 42.5%


In [ ]:
xgb_model = xgb.XGBClassifier(n_estimators=100,objective='multi:softmax')
train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size=0.2)
xgb_model.fit(train_X, train_y)
preds = xgb_model.predict(test_X)
acc_xgb = (preds == test_y).sum().astype(float) / len(preds)*100
print("XGBoost's prediction accuracy is: %3.2f" % (acc_xgb))


In [ ]:
!git clone https://github.com/philipperemy/keras-attention-mechanism

Cloning into 'keras-attention-mechanism'...
remote: Enumerating objects: 365, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 365 (delta 65), reused 63 (delta 27), pack-reused 243
Receiving objects: 100% (365/365), 4.15 MiB | 9.66 MiB/s, done.
Resolving deltas: 100% (179/179), done.


In [ ]:
!mv /content/keras-attention-mechanism/attention /content/

In [ ]:


from tensorflow.python.keras.layers import LSTM
from attention import Attention
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import PReLU
from tensorflow.keras.layers import Conv2D, LSTM
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten, Bidirectional
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam



In [34]:
Y = pd.get_dummies(df['label']).values
X=df.drop("label",axis=1)

Y.shape

(11540, 114)

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size=0.2)


In [ ]:
train_X=train_X.to_numpy()
test_X=test_X.to_numpy()

In [ ]:
126/3

42.0

In [ ]:
train_X=train_X.reshape(train_X.shape[0],42,3)
test_X=test_X.reshape(test_X.shape[0],42,3)

In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
model = Sequential()
model.add(LSTM(126, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True))
#model.add(Dropout(0.2))
# Second LSTM layer
model.add(LSTM(units=50, return_sequences=True))
#model.add(Dropout(0.2))
# Third LSTM layer
model.add(LSTM(units=50, return_sequences=True))
#model.add(Dropout(0.2))
# Fourth LSTM layer
model.add(LSTM(70, return_sequences=True))
#model.add(Dropout(0.3))
# The output layer
model.add(LSTM(units=50))
#model.add(Dropout(0.2))
model.add(Dense(114, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#model.compile(optimizer='adam',loss = tensorflow.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:

from tensorflow.keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='loss', patience=7, verbose=2)

# fit network
history = model.fit(train_X, train_y, epochs=120, batch_size=32, validation_preproc_data=(test_X, test_y), verbose=2, callbacks=[callback], shuffle=False)

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

# make a prediction
yhat = model.predict(test_X)
# calculate RMSE
rmse = np.sqrt(mean_squared_error(test_y, yhat))
print('Test RMSE: %.3f' % rmse)

# 1DCNN

In [23]:
import keras
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization, Dense
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.regularizers import l2
import seaborn as sns
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report

In [38]:
import keras
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization, Dense
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.regularizers import l2
import seaborn as sns
from tensorflow.python.keras.optimizer_v2.adam import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten, Conv2D
from tensorflow.keras.layers import Dense, Bidirectional
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [6]:


# NORMALIZE DATA
def NORMALIZE_DATA(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    X = (X - mean)/std


NORMALIZE_DATA(X)


In [47]:
train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size=0.2)


In [48]:
train_X=train_X.to_numpy()
test_X=test_X.to_numpy()

In [46]:
train_X.shape

(9232, 126, 1)

In [18]:
train_X=train_X.reshape(train_X.shape[0],42,3)
test_X=test_X.reshape(test_X.shape[0],42,3)

In [49]:
train_X=train_X.reshape(train_X.shape[0],126,1)
test_X=test_X.reshape(test_X.shape[0],126,1)

In [50]:

train_y = np.float64(train_y)
test_y = np.float64(test_y)
test_X = np.float64(test_X)
train_X = np.float64(train_X)

In [12]:
model = Sequential()
model.add(Conv1D(32, kernel_size=(3), activation='relu', input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Conv1D(64, (3), activation='relu'))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Dropout(0.2))

model.add(Conv1D(32, kernel_size=(3), activation='relu'))
model.add(Conv1D(128, (3), activation='relu'))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(114, activation='softmax'))

model.summary()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 40, 32)            320       
                                                                 
 conv1d_1 (Conv1D)           (None, 38, 64)            6208      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 19, 64)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 19, 64)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 17, 32)            6176      
                                                                 
 conv1d_3 (Conv1D)           (None, 15, 128)           12416     
                                                        

In [21]:



from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Set callback functions to early stop training and save the best model so far
checkpoint = [ModelCheckpoint(filepath='models_cnn1d.hdf5',monitor='val_accuracy',mode='max',save_best_only=True)]
callback = EarlyStopping(monitor='loss', patience=20, verbose=2)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=0.00001)

# fit network
history = model.fit(train_X, train_y, epochs=1200, batch_size=64, validation_data=(test_X, test_y), verbose=2, callbacks=[checkpoint,callback,reduce_lr], shuffle=False)

# summarize history for loss

Epoch 1/1200
145/145 - 8s - loss: 0.1811 - accuracy: 0.9387 - val_loss: 0.8396 - val_accuracy: 0.9155 - lr: 0.0010 - 8s/epoch - 54ms/step
Epoch 2/1200
145/145 - 7s - loss: 0.1747 - accuracy: 0.9404 - val_loss: 0.8121 - val_accuracy: 0.9181 - lr: 0.0010 - 7s/epoch - 48ms/step
Epoch 3/1200
145/145 - 8s - loss: 0.1766 - accuracy: 0.9417 - val_loss: 0.7908 - val_accuracy: 0.9185 - lr: 0.0010 - 8s/epoch - 52ms/step
Epoch 4/1200
145/145 - 6s - loss: 0.1825 - accuracy: 0.9380 - val_loss: 0.8226 - val_accuracy: 0.9159 - lr: 0.0010 - 6s/epoch - 40ms/step
Epoch 5/1200
145/145 - 6s - loss: 0.1789 - accuracy: 0.9409 - val_loss: 0.8144 - val_accuracy: 0.9207 - lr: 0.0010 - 6s/epoch - 38ms/step
Epoch 6/1200
145/145 - 6s - loss: 0.1916 - accuracy: 0.9377 - val_loss: 0.7833 - val_accuracy: 0.9211 - lr: 0.0010 - 6s/epoch - 44ms/step
Epoch 7/1200
145/145 - 6s - loss: 0.1709 - accuracy: 0.9407 - val_loss: 0.8206 - val_accuracy: 0.9190 - lr: 0.0010 - 6s/epoch - 44ms/step
Epoch 8/1200
145/145 - 6s - loss: 

In [ ]:
model.evaluate(test_X, test_y, batch_size=32)

126/126 [==============================] - 1s 6ms/step - loss: 2.5609 - accuracy: 0.6949


[2.5609049797058105, 0.6949025392532349]

In [26]:
!cp /content/models_cnn1d.hdf5 /content/drive/MyDrive

# LSTM model

In [51]:
# Neural network model
input_shape = (train_X.shape[1], train_X.shape[2])
optimizer = Adam(0.005, beta_1=0.1, beta_2=0.001, amsgrad=True)
n_classes = 114

model = Sequential()
model.add(Bidirectional(LSTM(126, return_sequences=True), input_shape=input_shape))
model.add(Bidirectional(LSTM(126, return_sequences=True)))
#model.add(Attention(126))
model.add(Dense(4*126))
model.add(Dense(4*126))
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

In [ ]:



from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Set callback functions to early stop training and save the best model so far
checkpoint = [ModelCheckpoint(filepath='models_cnn1d.hdf5',monitor='val_accuracy',mode='max',save_best_only=True)]
callback = EarlyStopping(monitor='loss', patience=20, verbose=2)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=0.00001)

# fit network
history = model.fit(train_X, train_y, epochs=1200, batch_size=64, validation_data=(test_X, test_y), verbose=2, callbacks=[checkpoint,callback,reduce_lr], shuffle=False)

# summarize history for loss

Epoch 1/1200
145/145 - 248s - loss: 4.4794 - acc: 0.1317 - val_loss: 4.0868 - val_acc: 0.1404 - lr: 0.0050 - 248s/epoch - 2s/step
Epoch 2/1200
145/145 - 228s - loss: 3.7112 - acc: 0.1980 - val_loss: 3.7913 - val_acc: 0.1902 - lr: 0.0050 - 228s/epoch - 2s/step
Epoch 3/1200
145/145 - 220s - loss: 3.4087 - acc: 0.2375 - val_loss: 3.6390 - val_acc: 0.2127 - lr: 0.0050 - 220s/epoch - 2s/step
Epoch 4/1200
145/145 - 225s - loss: 3.2322 - acc: 0.2626 - val_loss: 3.5112 - val_acc: 0.2288 - lr: 0.0050 - 225s/epoch - 2s/step
Epoch 5/1200
145/145 - 236s - loss: 3.1061 - acc: 0.2810 - val_loss: 3.4278 - val_acc: 0.2422 - lr: 0.0050 - 236s/epoch - 2s/step
Epoch 6/1200
145/145 - 258s - loss: 3.0054 - acc: 0.2940 - val_loss: 3.3627 - val_acc: 0.2487 - lr: 0.0050 - 258s/epoch - 2s/step
Epoch 7/1200
145/145 - 250s - loss: 2.9184 - acc: 0.3049 - val_loss: 3.3101 - val_acc: 0.2556 - lr: 0.0050 - 250s/epoch - 2s/step
Epoch 8/1200
145/145 - 254s - loss: 2.8369 - acc: 0.3159 - val_loss: 3.2501 - val_acc: 0.2

# KNN

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn_model = KNeighborsClassifier()

knn_pipe = Pipeline([
        ('sc', StandardScaler()),     
        ('knn', KNeighborsClassifier()) 
    ])

knn_param = {
        'knn__n_neighbors': [1,7], # usually odd numbers
        'knn__weights': ['uniform'],
        'knn__algorithm':['auto'],
        'knn__leaf_size':[10,20],
        'knn__p':[2,6,10]
    }

grid_knn = GridSearchCV(knn_pipe, 
                    param_grid = knn_param, 
                    n_jobs = -1, 
                    cv = 5,
                    refit = "accuracy_score",
                    verbose=3)

grid_knn.fit(X_train, y_train)


In [ ]:

prediction_knn = grid_knn.predict(X_test)

print(grid_knn.best_params_)
print('Score of best mlp_classifier :\t{}'.format(grid_knn.score(X_test, y_test)))
print(classification_report(y_test, prediction_knn))

{'knn__algorithm': 'auto', 'knn__leaf_size': 10, 'knn__n_neighbors': 1, 'knn__p': 2, 'knn__weights': 'uniform'}
Score of best mlp_classifier :	0.7839403973509934
                 precision    recall  f1-score   support

          A LOT       0.89      0.73      0.80        11
          ABUSE       1.00      0.81      0.90        16
         AFRAID       0.93      0.81      0.87        16
          AGREE       0.89      0.89      0.89        18
            ALL       0.60      0.67      0.63         9
          ANGRY       0.93      0.86      0.89        29
       ANYTHING       0.85      1.00      0.92        11
     APPRECIATE       0.56      0.56      0.56         9
            BAD       0.95      0.90      0.92        20
      BEAUTIFUL       0.90      0.60      0.72        15
         BECOME       0.73      0.73      0.73        11
            BED       0.46      0.67      0.55         9
          BORED       0.81      0.76      0.79        17
          BRING       0.71      0.75   